# Mô tả bài toán:
Yêu cầu xây dựng thuật toán để đưa ra giá cả phù hợp cho từng sản phẩm. Ta sử dụng thông tin sản phẩm do Mercari cung cấp, được người bán mô tả các chi tiết của sản phẩm như tên thương hiệu sản phẩm, tên sản phẩm, tình trạng sản phẩm để dưa ra giá phù hợp nhất.

# Xử lý dữ liệu

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Giải nén các file nén dữ liệu

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip


## Dùng thư viện pandas để đọc dữ liệu từ bài toán vào dạng table

In [ ]:
train_dataset = pd.read_table("train.tsv")
test_dataset = pd.read_table("test_stg2.tsv")

## Kiểm tra thông tin của tập dữ liệu train và test

In [ ]:
train_dataset.info()
test_dataset.info()

In [ ]:
print(train_dataset.shape)
print(test_dataset.shape)

* Tập train gồm 8 cột thông tin về sản phẩm và có tổng cộng 1482535 sản phẩm
* Tập test gồm 3460725 sản phẩm, có 7 cột thông tin và ta dự vào đó để tìm ra 'price' của sản phẩm
* Xuất ra thông tin về 10 sản phẩm đầu tiên của tập train và test

In [ ]:
train_dataset.head(10)

In [ ]:
test_dataset.head(10)

Ta loại bỏ các sản phẩm có price <= 0
Chia 10% tập train sau khi lọc thành tập validation_X không được huấn luyện để kiểm tra độ hiệu quả của mô hình trên tập train_X(90% của tập train sau khi lọc)

In [ ]:
train_dataset = train_dataset[train_dataset['price'] > 0].reset_index(drop = True)
train, validation = train_test_split(train_dataset, test_size = 0.1, random_state = 30)
test = test_dataset.copy()
print(train.shape)
print(validation.shape)
print(test.shape)

Kiểm tra số phần từ null của tập train_X và tập validation_X

In [ ]:
train.isnull().sum()

In [ ]:
validation.isnull().sum()

In [ ]:
test.isnull().sum()

Ta nhận thấy cột category_name và cột brand_name có rất nhiều giá trị null. Vì thế ta cần thay thế các giá trị null bằng "Missing"

In [ ]:
def fill(data):
    data['category_name'].fillna('Missing', inplace = True)
    data['brand_name'].fillna('Missing', inplace = True)
    data['item_description'].fillna('Missing', inplace = True)
    return data

In [ ]:
fill(train)
fill(validation)
fill(test)

Qua việc xuất ra thông tin của 10 sản phẩm đầu tiên của tập train và test, ta dễ nhận thấy cột category_name được chia thành 3 sub_category nhỏ và được chia cách nhau bởi kí tự '/'. Vì vậy ta tách category thành 3 category nhỏ để kiểm tra thông tin dễ dàng hơn

In [ ]:
def split_category(cate):
    try:
        sub_cate1, sub_cate2, sub_cate3 = cate.split("/")
        return sub_cate1, sub_cate2, sub_cate3
    except:
        return ("Missing", "Missing", "Missing")
    
def creat_category(data):
    data['sub_category1'], data['sub_category2'], data['sub_category3'] = zip(*data['category_name'].apply(lambda x: split_category(x)))

In [ ]:
creat_category(train)
creat_category(validation)
creat_category(test)

In [ ]:
test.head(10)

Ta kiểm tra phân phối của price trên tập train

In [ ]:
train.price.describe().apply(lambda x: format(x, 'f'))

Biểu diễn phân phối của biến price (gía cả) của các mặt hàng thông qua đồ thị

In [ ]:
sns.displot(np.log1p(train['price']))

# Tiền xử lý dữ liệu:
*      Dùng các phương pháp như : CountVectorizer, TfidfVectorizer, LabelBinarizer
*      Chuyển dữ liệu từ text (văn bản) sang các vector đặc trưng để máy tính có thể hiểu được.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import csr_matrix, hstack

In [ ]:
merge: pd.DataFrame = pd.concat([train, validation, test])
submission: pd.DataFrame = test[['test_id']]
n_train = train.shape[0]
n_validation = validation.shape[0]
n_test = test.shape[0]

In [ ]:
merge.info()
print(n_train, n_validation, n_test)

In [ ]:
NAME_MIN_DF = 10
MAX_FEATURES_ITEM_DESCRIPTION = 30000

def condition2vec(data):
    vectorizer = LabelBinarizer(sparse_output=True)
    vec = vectorizer.fit_transform(data)
    return vec


def shipping2vec(data):
    vectorizer = LabelBinarizer(sparse_output=True)
    vec = vectorizer.fit_transform(data)
    return vec


def brandname2vec(data):
    vectorizer = LabelBinarizer(sparse_output=True)
    vec = vectorizer.fit_transform(data)
    return vec

def name2vec(data):
    vectorizer = CountVectorizer(
        min_df=NAME_MIN_DF,                         
        ngram_range=(1, 2),                 
        stop_words='english'
    )
    vec = vectorizer.fit_transform(data)
    return vec

def description2vec(data):
    vectorizer = TfidfVectorizer(
        max_features=MAX_FEATURES_ITEM_DESCRIPTION,
        ngram_range=(1, 2),
        stop_words='english'
    )
    vec = vectorizer.fit_transform(data)
    return vec

def category2vec(data):
    vectorizer = CountVectorizer()
    data_vec = []
    
    for x in data: 
        category_vec = vectorizer.fit_transform(x)
        data_vec.append(category_vec)
    print('Convert Category ')
    return hstack(data_vec)

# data_vec = category2vec(data=(merge['sub_category1'], merge['sub_category2'], merge['sub_category3']))


def put_all_together(merge):
    res = hstack((
        name2vec(merge['name']),
        condition2vec(merge['item_condition_id']), 
        brandname2vec(merge['brand_name']), 
        shipping2vec(merge['shipping']), 
        description2vec(merge['item_description']), 
        category2vec((merge['sub_category1'], merge['sub_category2'], merge['sub_category3']))
    ))
    return res

In [ ]:
all_data = put_all_together(merge)
print(all_data.shape)

### Tách Dữ Liệu về các tập dữ liệu về train, validation, test

In [ ]:
all_data = all_data.tocsr()
train_X, validation_X, test_X = all_data[:n_train], all_data[n_train:n_train + n_validation], all_data[n_train + n_validation:]

### Tạo nhãn dữ liệu cho tập train

In [ ]:
train_Y = np.log1p(train["price"])
validation_Y = np.log1p(validation["price"])

### Khai báo các mô hình học máy

In [ ]:
from sklearn.linear_model import Ridge

### Tạo mô hình và huấn luyện dữ liệu

In [ ]:
model = Ridge(
    alpha=.5, 
    copy_X=True, 
    fit_intercept=True, 
    max_iter=100,
    normalize=False, 
    random_state=101, 
    solver='auto', 
    tol=0.01
)
model.fit(train_X, train_Y)

### Hàm đánh giá chất lương mô hình với tập dữ liêu

In [ ]:
def rmsle(y, y0):
    assert len(y) == len(y0)
    return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y0), 2)))

In [ ]:
val_pred = model.predict(validation_X)
val_pred = np.expm1(val_pred)
print(rmsle(val_pred, validation["price"]))

In [ ]:
test_pred = model.predict(test_X)
submission['price'] = np.expm1(test_pred)

In [ ]:
 submission.to_csv("submission.csv", index=False)